In [9]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 432 kB 4.3 MB/s 
     |████████████████████████████████| 13.7 MB 26.5 MB/s 
     |████████████████████████████████| 3.5 MB 16.4 MB/s 
     |████████████████████████████████| 1.2 MB 50.7 MB/s 
     |████████████████████████████████| 622 kB 63.4 MB/s 
     |████████████████████████████████| 351 kB 65.5 MB/s 
     |████████████████████████████████| 89 kB 7.1 MB/s 
     |████████████████████████████████| 6.0 MB 40.0 MB/s 
     |████████████████████████████████| 1.6 MB 49.8 MB/s 
     |████████████████████████████████| 451 kB 52.0 MB/s 
     |████████████████████████████████| 628 kB 31.4 MB/s 
     |████████████████████████████████| 42 kB 932 kB/s 
     |████████████████████████████████| 181 kB 59.3 MB/s 
     |████████████████████████████████| 10.1 MB 25.0 MB/s 
     |████████████████████████████████| 174 kB 47.0 MB/s 
     |████████████████████████████████| 5.9 MB 30.5 MB/s 
     |████████████████████████████████| 895 kB 52.3 MB/s 
     |███████████

# Dataset

In [1]:
#Datos bd
import pandas as pd

df_msg = pd.read_csv('Message.csv')
df_cat = pd.read_csv('Category.csv')
df_user = pd.read_csv('User.csv') 

df_userMes = pd.read_csv('UserMessage.csv') 
df_mesCat = pd.read_csv('MessageCategory.csv') 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## EDA
In this section we are going to use only data_pp column.

In [2]:
df_msg= df_msg.dropna(subset=['data_pp'])

In [3]:
final = pd.merge(df_msg, df_mesCat, on='id_message')
final = pd.merge(final, df_cat, on='id_category')
final = pd.merge(final, df_userMes, on='id_message')

In [4]:
df = final.copy()

In [5]:
#df = df.drop(columns=['id_message', 'dataset','timestamp','data','id_forwarded','emojis','id_reply'])
df = df.drop(columns=['dataset','timestamp','data','id_forwarded','emojis','id_reply'])

In [7]:
df = df.drop_duplicates('data_pp')

In [8]:
df

,id_message,data_pp,id_category,name,target,is_general,is_women,is_union,is_geographic,is_thematic,deleted,hacked,id_user
0,3AE38F3CD29954C10309,persona 3ro edad x provoste diciembre x kast p...,0,Atrévete con kast🇨🇱🗣️,Kast,True,False,False,False,False,False,False,8064210797453835407
1,8D36961277BD5A9F9D245F88A2E67296,mismo cecilia comunismo pobres cubano,0,Atrévete con kast🇨🇱🗣️,Kast,True,False,False,False,False,False,False,8064210797453835041
2,8862501799EE8D65AC8866BB0CEAB1E6,siiiiiiiiiiiiiiii,0,Atrévete con kast🇨🇱🗣️,Kast,True,False,False,False,False,False,False,8064210797453835000
3,A69F2E0B649A1E1FF4F95DFFFB9CEB09,parisi,0,Atrévete con kast🇨🇱🗣️,Kast,True,False,False,False,False,False,False,8064210797453835719
4,28814E7C6F1C30F858A385549F2D62D5,boric,0,Atrévete con kast🇨🇱🗣️,Kast,True,False,False,False,False,False,False,8064210797453835719
...,...,...,...,...,...,...,...,...,...,...,...,...,...
552308,519,firma cuidad especifico,4611694699617657433,Partido de la Gente,Parisi,True,False,False,False,False,False,False,43200896983991160
552312,524,partido año,4611694699617657433,Partido de la Gente,Parisi,True,False,False,False,False,False,False,43200896983991153
552313,525,buena listo,4611694699617657433,Partido de la Gente,Parisi,True,False,False,False,False,False,False,43200896983991160
552314,526,yapo ayseninos pila,4611694699617657433,Partido de la Gente,Parisi,True,False,False,False,False,False,False,43200896983991157


# Idea 2:Zero-shot classification con los títulos de wsp 

In [10]:
from transformers import pipeline

In [11]:
generator = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

Downloading:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/463 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [17]:
g=[]; p=[]; m=[]; b=[]; a=[]; k = []; s = []
labels = ["geografia", "politica", "kast","mujer","boric"]


In [15]:
data =   df_cat['name'].tolist()

In [18]:
for i in data:
  aux = generator(i, labels, multi_label=True)
  
  s.append(aux['sequence'])
  for j in range(len(aux['labels'])):
    if str(aux['labels'][j]) == 'geografia':
      g.append(aux['scores'][j])
    if str(aux['labels'][j]) == 'politica':
      p.append(aux['scores'][j])
    if str(aux['labels'][j]) == 'kast':
      k.append(aux['scores'][j])
    if str(aux['labels'][j] ) == 'mujer':
      m.append(aux['scores'][j])
    if str(aux['labels'][j] )== 'boric':
      b.append(aux['scores'][j])


In [20]:

df_ej = pd.DataFrame(data={'Grupo': s, 'Politica':p, 'Boric':b,'Kast':k, 'Mujer':m, 'Geografia':g})

In [21]:
df_ej

,Grupo,Politica,Boric,Kast,Mujer,Geografia
0,Atrévete con kast🇨🇱🗣️,0.191491,0.164737,0.959168,0.533871,0.043302
1,1 Kast Presidente🇨🇱,0.965231,0.149234,0.982421,0.624683,0.006644
2,Region Coquimbo con KAST,0.703234,0.176986,0.979289,0.280412,0.982971
3,Atrévete Atacama,0.058237,0.206028,0.117036,0.288212,0.969359
4,Atrévete Tarapacá,0.850905,0.176341,0.070293,0.627963,0.965510
...,...,...,...,...,...,...
545,Oposición por la Patria,0.951150,0.377020,0.422548,0.512206,0.040977
546,RechazoConvención x🇨🇱,0.867197,0.302297,0.083630,0.257929,0.004170
547,PATRIA Y LIBERTAD 🇨🇱,0.907534,0.076159,0.033988,0.120522,0.012742
548,JOVENES DE DERECHA 🤝🏻💪🏻🧐,0.711462,0.530986,0.148323,0.047070,0.921898


In [22]:
df_lab_binary = df_ej.copy() 


In [23]:
for i in df_lab_binary.columns[1:]:
  df_lab_binary.loc[df_lab_binary[i] >= 0.7, i] = 1
  df_lab_binary.loc[df_lab_binary[i] < 0.7, i] =  0

In [24]:
df_lab_binary

,Grupo,Politica,Boric,Kast,Mujer,Geografia
0,Atrévete con kast🇨🇱🗣️,0.0,0.0,1.0,0.0,0.0
1,1 Kast Presidente🇨🇱,1.0,0.0,1.0,0.0,0.0
2,Region Coquimbo con KAST,1.0,0.0,1.0,0.0,1.0
3,Atrévete Atacama,0.0,0.0,0.0,0.0,1.0
4,Atrévete Tarapacá,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
545,Oposición por la Patria,1.0,0.0,0.0,0.0,0.0
546,RechazoConvención x🇨🇱,1.0,0.0,0.0,0.0,0.0
547,PATRIA Y LIBERTAD 🇨🇱,1.0,0.0,0.0,0.0,0.0
548,JOVENES DE DERECHA 🤝🏻💪🏻🧐,1.0,0.0,0.0,0.0,1.0


# Idea 3: Clasificación de texto utilizando los targets de los títulos

* Binary Relevance
* Classifier Chains
* Label Powerset

In [25]:
df_cat

,id_category,name,target,is_general,is_women,is_union,is_geographic,is_thematic,deleted,hacked
0,0,Atrévete con kast🇨🇱🗣️,Kast,True,False,False,False,False,False,False
1,1,1 Kast Presidente🇨🇱,Kast,True,False,False,False,False,False,False
2,2,Region Coquimbo con KAST,Kast,False,False,False,True,False,False,False
3,3,Atrévete Atacama,Kast,False,False,False,True,False,False,False
4,4,Atrévete Tarapacá,Kast,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...
545,545,Oposición por la Patria,Kast,True,False,False,False,False,False,False
546,546,RechazoConvención x🇨🇱,Kast,False,False,False,False,True,False,False
547,547,PATRIA Y LIBERTAD 🇨🇱,Kast,False,False,False,False,True,False,False
548,548,JOVENES DE DERECHA 🤝🏻💪🏻🧐,Kast,False,False,False,False,True,False,False


In [26]:
y = df_cat[['is_general',	'is_women',	'is_union',	'is_geographic',	'is_thematic']].astype(int)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(data).toarray()

In [28]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y.values,test_size=0.1,random_state=42)

### For this part it is necessary to install the following sklearn version

In [ ]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==0.24.1

Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Successfully uninstalled scikit-learn-1.0.2
     |████████████████████████████████| 22.3 MB 1.3 MB/s 


In [ ]:
from skmultilearn.adapt import MLkNN

mlknn_classifier = MLkNN()
mlknn_classifier.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass n_neighbors=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


MLkNN()

In [ ]:
mlkn_pred = mlknn_classifier.predict(X_test)

In [ ]:
accuracy_score(y_test,mlkn_pred)

0.4727272727272727

## BinaryRelevance


In [29]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB,MultinomialNB

In [30]:
binary_rel = BinaryRelevance(MultinomialNB())

In [31]:
binary_rel.fit(X_train,y_train)

BinaryRelevance(classifier=MultinomialNB(), require_dense=[True, True])

In [32]:
br_prediction = binary_rel.predict(X_test)

In [36]:
from sklearn.metrics import accuracy_score,hamming_loss,classification_report
print(accuracy_score(y_test,br_prediction))
print(hamming_loss(y_test,br_prediction))

0.5818181818181818
0.12


## ClassifierChain

In [37]:
from sklearn.tree import DecisionTreeClassifier
from skmultilearn.problem_transform import ClassifierChain

In [38]:
cl_ch = ClassifierChain(DecisionTreeClassifier())

In [39]:
cl_ch.fit(X_train,y_train)

ClassifierChain(classifier=DecisionTreeClassifier(), require_dense=[True, True])

In [41]:
cl_ch_prediction = cl_ch.predict(X_test)

In [42]:
from sklearn.metrics import accuracy_score,hamming_loss,classification_report
print(accuracy_score(y_test,cl_ch_prediction))
print(hamming_loss(y_test,cl_ch_prediction))

0.6181818181818182
0.13454545454545455


## LabelPowerset

In [44]:
from skmultilearn.problem_transform import LabelPowerset
from sklearn.ensemble import RandomForestClassifier

rf = LabelPowerset(RandomForestClassifier())

rf.fit(X_train, y_train)

predictions_lp = rf.predict(X_test)

In [45]:
from sklearn.metrics import accuracy_score,hamming_loss,classification_report
print(accuracy_score(y_test,predictions_lp))
print(hamming_loss(y_test,predictions_lp))

0.6545454545454545
0.11272727272727273


## OneVsRestClassifier

In [46]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier

In [47]:
clf_or = OneVsRestClassifier( LogisticRegression(penalty='l1', C=1, dual=False, solver='liblinear'))
clf_or.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1, penalty='l1',
                                                 solver='liblinear'))

In [48]:
clf_or_predicted = clf_or.predict(X_test)
clf_or_predicted_scores = clf_or.decision_function(X_test)

In [49]:
from sklearn.metrics import accuracy_score,hamming_loss,classification_report
print(accuracy_score(y_test,clf_or_predicted))
print(hamming_loss(y_test,clf_or_predicted))

0.6363636363636364
0.11636363636363636
